# 尝试自动提取特征

In [43]:
import tensorflow as tf
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
import shutil
import sklearn
import cv2
import csv
import math
#import image
from skimage.io import imread
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.image as mpimg
#导入图表库
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import applications
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19

from keras.applications.vgg19 import preprocess_input

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from keras.utils import plot_model
# PCA 主成分分析
from sklearn.decomposition import PCA
from keras.callbacks import TensorBoard
print("hello tensorflow")

hello tensorflow


##  图像特征提取

### 拷贝 keras 内置经典网络

预训练的vgg-19

In [3]:
model_vgg19 = applications.VGG19(
    # 这里是利用预训练的模型来做特征提取
    # 因此我们不需要顶层的分类器网络部分的权重，只需要使用到训练好的卷积基。
    # 这也就是VGG19参数中include_top=False的含义
    include_top=False,
    weights='../../emotionData/vgg-19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
)
model_vgg19.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


## 网络构建

In [44]:
model = VGG19(weights='imagenet', include_top=True)


#model = keras.Sequential()
#model.add(model_vgg19)

# output = keras.layers.Conv2D(512,kernel_size=1,strides=1,padding='same',activation="sigmoid", name="convolution_1d_layer")

# model.add(output)
# dropout
# model.add(layers.Dropout(0.3))

# 对二维信号进行 平均值池化
# model.add(layers.GlobalAveragePooling2D())

# 2的任何次方（256 也可以）
# model.add(layers.Dense(512, activation='relu'))

# 打印模型概况
print(model.summary())

574717952/574710816 [==============================] - 1303s 2us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2

### 特征提取

In [30]:
imagesDir = '../../emotionData/image/jpg/'
#os.listdir(imagesDir)    

Found 0 images belonging to 0 classes.


In [73]:
list = []
del list[:]

for num in range(1,501):
    img_path = imagesDir + str(num) + '.jpg'
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))  # 加载图像，归一化大小
    x = image.img_to_array(img)   # 序列化
    x = np.expand_dims(x, axis=0)  # 展开
    x = preprocess_input(x)       # 预处理到0～1
    out = model.predict(x)  # 预测结果，1000维的向量
    out = np.insert(out, 0, num)
    print(out.shape)   # (1,1000)
    list.append(out)

test = pd.DataFrame(data=list)
test.to_csv('C:\\Users\\lab233\\Desktop\\emotionData\\image\\feature\\feature.csv')

../../emotionData/image/jpg/1.jpg
(1001,)
../../emotionData/image/jpg/2.jpg
(1001,)
../../emotionData/image/jpg/3.jpg
(1001,)
../../emotionData/image/jpg/4.jpg
(1001,)
../../emotionData/image/jpg/5.jpg
(1001,)
../../emotionData/image/jpg/6.jpg
(1001,)
../../emotionData/image/jpg/7.jpg
(1001,)
../../emotionData/image/jpg/8.jpg
(1001,)
../../emotionData/image/jpg/9.jpg
(1001,)
../../emotionData/image/jpg/10.jpg
(1001,)
../../emotionData/image/jpg/11.jpg
(1001,)
../../emotionData/image/jpg/12.jpg
(1001,)
../../emotionData/image/jpg/13.jpg
(1001,)
../../emotionData/image/jpg/14.jpg
(1001,)
../../emotionData/image/jpg/15.jpg
(1001,)
../../emotionData/image/jpg/16.jpg
(1001,)
../../emotionData/image/jpg/17.jpg
(1001,)
../../emotionData/image/jpg/18.jpg
(1001,)
../../emotionData/image/jpg/19.jpg
(1001,)
../../emotionData/image/jpg/20.jpg
(1001,)
../../emotionData/image/jpg/21.jpg
(1001,)
../../emotionData/image/jpg/22.jpg
(1001,)
../../emotionData/image/jpg/23.jpg
(1001,)
../../emotionData/im

(1001,)
../../emotionData/image/jpg/190.jpg
(1001,)
../../emotionData/image/jpg/191.jpg
(1001,)
../../emotionData/image/jpg/192.jpg
(1001,)
../../emotionData/image/jpg/193.jpg
(1001,)
../../emotionData/image/jpg/194.jpg
(1001,)
../../emotionData/image/jpg/195.jpg
(1001,)
../../emotionData/image/jpg/196.jpg
(1001,)
../../emotionData/image/jpg/197.jpg
(1001,)
../../emotionData/image/jpg/198.jpg
(1001,)
../../emotionData/image/jpg/199.jpg
(1001,)
../../emotionData/image/jpg/200.jpg
(1001,)
../../emotionData/image/jpg/201.jpg
(1001,)
../../emotionData/image/jpg/202.jpg
(1001,)
../../emotionData/image/jpg/203.jpg
(1001,)
../../emotionData/image/jpg/204.jpg
(1001,)
../../emotionData/image/jpg/205.jpg
(1001,)
../../emotionData/image/jpg/206.jpg
(1001,)
../../emotionData/image/jpg/207.jpg
(1001,)
../../emotionData/image/jpg/208.jpg
(1001,)
../../emotionData/image/jpg/209.jpg
(1001,)
../../emotionData/image/jpg/210.jpg
(1001,)
../../emotionData/image/jpg/211.jpg
(1001,)
../../emotionData/image/

(1001,)
../../emotionData/image/jpg/377.jpg
(1001,)
../../emotionData/image/jpg/378.jpg
(1001,)
../../emotionData/image/jpg/379.jpg
(1001,)
../../emotionData/image/jpg/380.jpg
(1001,)
../../emotionData/image/jpg/381.jpg
(1001,)
../../emotionData/image/jpg/382.jpg
(1001,)
../../emotionData/image/jpg/383.jpg
(1001,)
../../emotionData/image/jpg/384.jpg
(1001,)
../../emotionData/image/jpg/385.jpg
(1001,)
../../emotionData/image/jpg/386.jpg
(1001,)
../../emotionData/image/jpg/387.jpg
(1001,)
../../emotionData/image/jpg/388.jpg
(1001,)
../../emotionData/image/jpg/389.jpg
(1001,)
../../emotionData/image/jpg/390.jpg
(1001,)
../../emotionData/image/jpg/391.jpg
(1001,)
../../emotionData/image/jpg/392.jpg
(1001,)
../../emotionData/image/jpg/393.jpg
(1001,)
../../emotionData/image/jpg/394.jpg
(1001,)
../../emotionData/image/jpg/395.jpg
(1001,)
../../emotionData/image/jpg/396.jpg
(1001,)
../../emotionData/image/jpg/397.jpg
(1001,)
../../emotionData/image/jpg/398.jpg
(1001,)
../../emotionData/image/

## 音频特征提取

## 特征融合

## 训练模型

#### 定义模型

#### 训练

## 预测

## 计算均方差